In [2]:
import mysql
from mysql import connect_mysql

conn = connect_mysql()

In [3]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

href = "https://www.setn.com/ViewAll.aspx?PageGroupID="
number = {5: '國際', 6: '政治', 41: '社會', 4: '生活', 65: '健康', 34: '運動', 12: '汽車', 2: '財經', 7: '科技'}

for i in number.keys():
  r = requests.get(href + str(i))
  soup = bs(r.text, "lxml")
  a = soup.find("div", {"class": "row NewsList"})
  for title_temp in (a.findAll("a", {"class": "gt"})):
    #初始化
    content = ''

    content_url = 'https://www.setn.com' + title_temp['href']
    news_id = content_url.split('=', 1)[1]
    news_id = news_id.split('&', 1)[0]
    print('三立 ' + news_id + ' ' + number[i])

    # 抓取內容
    r2 = requests.get(content_url)
    soup2 = bs(r2.text, "lxml")
    a2 = soup2.find("div", {"id": "Content1"})
    #-----------------------------------------------------------
    #標題
    title2 = soup2.find("title").text
    title2 = title2.split("	")[1]
    title = title2.split(" | ")[0]

    classification = title2.split(" | ")[1].split(" | ")[0]
    
    print(title)
    print('三立 ' + news_id + ' ' + classification)
    #-----------------------------------------------------------

    # 日期
    date = soup2.find("div", {"class": "page-title-text"})
    date = date.find('time', {'class': 'page-date'})
    date = date.text
    print(date)
    print(content_url)
    for content_temp in (a2.findAll("p")):
      if(('▲' in content_temp.text)):     #跳過圖片註解
        continue
      if((('/' in content_temp.text) or ('／' in content_temp.text)) and ('報導' in content_temp.text)):
        continue
      content += content_temp.text
      #print(content)

    # 確認是否已在DB中
    with conn.cursor() as cursor:
      sql = 'SELECT News_id FROM original WHERE News_id = %s AND Provenance = \'三立\''
      cursor.execute(sql, (news_id))
      result = cursor.fetchall()
      check = None
      for row in result:
        check = row[0]

    # 若已在DB中則跳過
    if(str(check) == news_id):
      continue
    with conn.cursor() as cursor:
        # 新增資料SQL語法
        command = 'INSERT INTO original VALUES(%s, %s, %s, %s, %s, %s, %s)'
        cursor.execute(command, (news_id, date, title, content_url, classification, '三立', content))
        # 儲存變更
    conn.commit()

    with conn.cursor() as cursor:
        # 新增資料SQL語法
        command = 'INSERT INTO processed_data (News_id, Provenance) VALUES(%s, %s)'
        cursor.execute(command, (news_id, '三立'))
        # 儲存變更
    conn.commit()
    

    print('--------------\n')

三立 1006800 國際
比利時長者施打第3劑　特別脆弱族群研擬打第4劑
三立 1006800 國際
2021/10/02 23:44:00
https://www.setn.com/News.aspx?NewsID=1006800&utm_source=setn.com&utm_medium=viewall&utm_campaign=viewallnews
三立 1006795 國際
杜特蒂宣布退出政壇　薩拉支持者轉向小馬可仕
三立 1006795 國際
2021/10/02 23:33:00
https://www.setn.com/News.aspx?NewsID=1006795&utm_source=setn.com&utm_medium=viewall&utm_campaign=viewallnews
三立 1006794 國際
首都不例外！北京節約用電暫停十一燈光秀
三立 1006794 國際
2021/10/02 23:22:00
https://www.setn.com/News.aspx?NewsID=1006794&utm_source=setn.com&utm_medium=viewall&utm_campaign=viewallnews
三立 1006793 國際
美國反墮胎法引爭議　女性大遊行走上街頭
三立 1006793 國際
2021/10/02 23:11:00
https://www.setn.com/News.aspx?NewsID=1006793&utm_source=setn.com&utm_medium=viewall&utm_campaign=viewallnews
三立 1006797 國際
1天20架共機擾台　學者：殲16具一定夜航能力
三立 1006797 國際
2021/10/02 22:01:00
https://www.setn.com/News.aspx?NewsID=1006797&utm_source=setn.com&utm_medium=viewall&utm_campaign=viewallnews
三立 1006790 國際
中國遭「空襲」？鄭州無人機失控墜落　群眾逃難畫面曝光
三立 1006790 國際
2021/10/02 21:31:00
https://www.setn.com/News